In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/07/26 10:20:51 WARN Utils: Your hostname, Lizs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.57 instead (on interface en0)
23/07/26 10:20:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/26 10:20:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import wget
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("Read S3 Data").getOrCreate()

# URL of the S3 bucket file
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Download the file using wget
local_file_path = "home_sales_revised.csv"
wget.download(url, local_file_path)

# Read the downloaded file into a DataFrame
df = spark.read.csv(local_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()


23/07/26 10:20:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


 57% [...............................                       ] 1581056 / 2747874

100% [......................................................] 2747874 / 2747874

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")
# Example SQL query on the temporary view
query_result = spark.sql("SELECT id, price, bedrooms FROM home_sales WHERE bedrooms >= 3")

# Show the result of the query
query_result.show()


+--------------------+------+--------+
|                  id| price|bedrooms|
+--------------------+------+--------+
|f8a53099-ba1c-47d...|936923|       4|
|e0726d4d-d595-407...|424418|       3|
|e81aacfe-17fe-46b...|181925|       3|
|2ed8d509-7372-46d...|258710|       3|
|f876d86f-3c9f-42b...|167864|       3|
|941bad30-eb49-4a7...|229896|       3|
|dd61eb34-6589-4c0...|210247|       3|
|ea620c7b-c2f7-4c6...|437958|       3|
|f233cb41-6f33-4b0...|437375|       4|
|0cfe57f3-28c2-472...|308313|       3|
|4566cd2a-ac6e-435...|177541|       3|
|50873b8b-92b8-44c...|268554|       3|
|e0fc52aa-c349-4ba...|202790|       3|
|0be5f2fd-e2ca-46c...|189137|       3|
|2dc662fe-57b8-44b...|417215|       4|
|e252c4ce-b5b2-4ef...|168463|       3|
|a16c1fd3-50c3-4e7...|372151|       3|
|d715f295-2fbf-4e9...|391574|       3|
|a18515a2-86f3-46b...|419543|       3|
|d5f2062a-8950-417...|305980|       4|
+--------------------+------+--------+
only showing top 20 rows



In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

# SQL query to calculate the average price for a four-bedroom house sold in each year
query = """
SELECT YEAR(date) as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
"""

# Execute the query
result = spark.sql(query)

# Show the result
result.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

df.createOrReplaceTempView("home_sales")

query = """
SELECT YEAR(to_date(CAST(date_built AS STRING), 'yyyy')) as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year
ORDER BY year
"""


result = spark.sql(query)

result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [8]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

df.createOrReplaceTempView("home_sales")


query = """
SELECT YEAR(to_date(CAST(date_built AS STRING), 'yyyy')) as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY year
ORDER BY year
"""

result = spark.sql(query)

result.show()

+----+---------+
|year|avg_price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

df.createOrReplaceTempView("home_sales")


query = """
SELECT ROUND(AVG(price), 2) as avg_price, ROUND(AVG(view), 2) as avg_view
FROM home_sales
WHERE price >= 350000
"""


result = spark.sql(query)

result.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|   32.26|
+---------+--------+

--- 3.314018249511719e-05 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
df.createOrReplaceTempView("home_sales")

# Cache the 'home_sales' DataFrame
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

df.createOrReplaceTempView("home_sales")

# Uncached Query Execution
start_time_uncached = time.time()

# SQL query to filter out the view ratings with average price greater than or equal to $350,000
query_uncached = """
SELECT ROUND(AVG(view), 2) as avg_view
FROM home_sales
WHERE price >= 350000
"""

# Execute the query without caching
result_uncached = spark.sql(query_uncached)

end_time_uncached = time.time()
runtime_uncached = end_time_uncached - start_time_uncached

# Cached Query Execution
start_time_cached = time.time()

# Cache the 'home_sales' DataFrame
spark.sql("CACHE TABLE home_sales")

# SQL query to filter out the view ratings with average price >= or equal to $350k
query_cached = """
SELECT ROUND(AVG(view), 2) as avg_view
FROM home_sales
WHERE price >= 350000
"""

# Execute the query with caching
result_cached = spark.sql(query_cached)

end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached

# Show the results and runtimes
print("Uncached Result:")
result_uncached.show()

print("Uncached Query Runtime:", runtime_uncached, "seconds")

print("\nCached Result:")
result_cached.show()

print("Cached Query Runtime:", runtime_cached, "seconds")

# print("--- %s seconds ---" % (time.time() - start_time))


23/07/26 10:30:05 WARN CacheManager: Asked to cache already cached data.


Uncached Result:
+--------+
|avg_view|
+--------+
|   32.26|
+--------+

Uncached Query Runtime: 0.025943756103515625 seconds

Cached Result:
+--------+
|avg_view|
+--------+
|   32.26|
+--------+

Cached Query Runtime: 0.21535825729370117 seconds


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

# partitionBy & write to Parquet format
df.write.partitionBy("date_built").parquet("partitioned_home_sales.parquet")


In [18]:
# 11. Read the formatted parquet data.
df_partitioned = spark.read.parquet("partitioned_home_sales.parquet")
df_partitioned.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [19]:
# 12. Create a temporary table for the parquet data.
df_partitioned.createOrReplaceTempView("parquet_home_sales")


In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

df_partitioned.createOrReplaceTempView("parquet_home_sales")

# Parquet DataFrame Query Execution (Without Caching)
start_time_parquet = time.time()

# SQL query to filter out the view ratings with average price greater than or equal to $350,000
query_parquet = """
SELECT ROUND(AVG(view), 2) as avg_view
FROM parquet_home_sales
WHERE price >= 350000
"""

# Execute the query on the Parquet DataFrame
result_parquet = spark.sql(query_parquet)

end_time_parquet = time.time()
runtime_parquet = end_time_parquet - start_time_parquet

# Show the Parquet DataFrame result and runtime
print("Parquet Result:")
result_parquet.show()

print("Parquet Query Runtime:", runtime_parquet, "seconds")

# Cached Query Execution (Assuming you have already cached the DataFrame as shown in previous answers)
start_time_cached = time.time()

# SQL query to filter out the view ratings with average price greater than or equal to $350,000
query_cached = """
SELECT ROUND(AVG(view), 2) as avg_view
FROM home_sales
WHERE price >= 350000
"""

# Execute the query on the cached DataFrame
result_cached = spark.sql(query_cached)

end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached

# Show the cached result and runtime
print("\nCached Result:")
result_cached.show()

print("Cached Query Runtime:", runtime_cached, "seconds")






# start_time = time.time()
# print("--- %s seconds ---" % (time.time() - start_time))

Parquet Result:
+--------+
|avg_view|
+--------+
|   32.26|
+--------+

Parquet Query Runtime: 0.049463748931884766 seconds

Cached Result:
+--------+
|avg_view|
+--------+
|   32.26|
+--------+

Cached Query Runtime: 0.04512834548950195 seconds


In [21]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [22]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

# fun way to print the result
if is_cached:
    print("'home_sales' DataFrame is still cached.")
else:
    print("'home_sales' DataFrame is not cached.")

'home_sales' DataFrame is not cached.
